In [3]:
import random
import math

# ----- Dataset (5 cities with coordinates) -----
cities = {
    "A": (0, 0),
    "B": (1, 3),
    "C": (4, 3),
    "D": (6, 1),
    "E": (3, 0)
}
city_list = list(cities.keys())

# ----- Helper Functions -----

def distance(city1, city2):
    x1, y1 = cities[city1]
    x2, y2 = cities[city2]
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def fitness(chromosome):
    dist = 0
    for i in range(len(chromosome) - 1):
        dist += distance(chromosome[i], chromosome[i+1])
    dist += distance(chromosome[-1], chromosome[0])  # return to start
    return 1 / dist  # smaller distance = higher fitness

def generate_population(size):
    population = []
    for _ in range(size):
        chromosome = random.sample(city_list, len(city_list))  # permutation
        population.append(chromosome)
    return population

def roulette_wheel_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    pick = random.uniform(0, total_fitness)
    current = 0
    for chromosome, fit in zip(population, fitness_scores):
        current += fit
        if current >= pick:
            return chromosome

def crossover(parent1, parent2):
    # Order Crossover (OX)
    size = len(parent1)
    p1, p2 = sorted(random.sample(range(size), 2))
    
    slice_ = parent1[p1:p2]
    child1 = slice_ + [c for c in parent2 if c not in slice_]
    child2 = parent2[p1:p2] + [c for c in parent1 if c not in parent2[p1:p2]]
    return child1, child2

def mutate(chromosome):
    i, j = random.sample(range(len(chromosome)), 2)
    chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
    return chromosome

def best_chromosome(population):
    best = population[0]
    best_fit = fitness(best)
    for c in population:
        fit = fitness(c)
        if fit > best_fit:
            best = c
            best_fit = fit
    return best, 1 / best_fit  # return tour and its distance

# ----- Genetic Algorithm -----

def genetic_algorithm(pop_size=10, generations=100, Pc=0.9, Pm=0.1):
    population = generate_population(pop_size)
    
    for _ in range(generations):
        fitness_scores = [fitness(c) for c in population]
        new_population = []
        
        while len(new_population) < pop_size:
            # Selection
            parent1 = roulette_wheel_selection(population, fitness_scores)
            parent2 = roulette_wheel_selection(population, fitness_scores)
            
            # Crossover
            if random.random() < Pc:
                child1, child2 = crossover(parent1, parent2)
            else:
                child1, child2 = parent1[:], parent2[:]
            
            # Mutation
            if random.random() < Pm:
                child1 = mutate(child1)
            if random.random() < Pm:
                child2 = mutate(child2)
            
            new_population.append(child1)
            new_population.append(child2)
        
        population = new_population[:pop_size]
    
    return best_chromosome(population)


# ----- Run the GA -----
best_tour, best_distance = genetic_algorithm(pop_size=20, generations=200, Pc=0.9, Pm=0.2)

print("Best Tour:", best_tour)
print("Shortest Distance:", round(best_distance, 2))


Best Tour: ['E', 'A', 'B', 'D', 'C']
Shortest Distance: 17.54
